<a href="https://colab.research.google.com/github/davide-sonzini/Flight_price_map/blob/main/Flight_price_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install amadeus pandas

import pandas as pd
import time
import datetime
from amadeus import Client, ResponseError
from google.colab import files
import random

# -------------------------------
# CONFIG
# -------------------------------
API_KEY = "ligX17GEhAERS34XjfmMW6rpG1c5uHZu"       # Replace with your Amadeus test GET API key
API_SECRET = "2jWVw0p9UA72Ykke" # Replace with your Amadeus secret
ORIGIN_AIRPORT = "MXP"
DEPARTURE_DATE = (datetime.date.today() + datetime.timedelta(days=30)).strftime("%Y-%m-%d")
AUTOSAVE_INTERVAL = 50  # Save after every 50 airports
SUBSET_SIZE = 1000       # Number of airports to randomly select

# -------------------------------
# INIT CLIENT
# -------------------------------
amadeus = Client(client_id=API_KEY, client_secret=API_SECRET)

# -------------------------------
# LOAD AND SUBSET AIRPORTS
# -------------------------------
airports = pd.read_csv('airports.csv')

if len(airports) > SUBSET_SIZE:
    airports_subset = airports.sample(n=SUBSET_SIZE, random_state=42).reset_index(drop=True)
else:
    airports_subset = airports.copy()

# Optional: save subset for reference
airports_subset.to_csv('airports_subset_1000.csv', index=False)

print(f"Random subset of {len(airports_subset)} airports created.")

# -------------------------------
# PRICE BUCKET / COLOR FUNCTION
# -------------------------------
def assign_bucket_and_color(price):
    if price is None:
        return "No Data", "#CCCCCC"
    elif price < 100:
        return "<100", "#0000FF"        # Blue
    elif price < 200:
        return "100-200", "#00FF00"    # Green 1
    elif price < 400:
        return "200-400", "#008000"    # Green 2
    elif price < 800:
        return "400-800", "#FFFF00"    # Yellow
    elif price < 1000:
        return "800-1000", "#FFA500"   # Orange
    else:
        return ">1000", "#FF0000"      # Red

# -------------------------------
# FUNCTION TO GET CHEAPEST FLIGHT
# -------------------------------
def get_cheapest_flight(origin, destination, date):
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin,
            destinationLocationCode=destination,
            departureDate=date,
            adults=1,
            max=5
        )
        data = response.data
        if not data:
            return None
        prices = [float(offer['price']['total']) for offer in data if 'price' in offer]
        return min(prices) if prices else None
    except ResponseError as e:
        print(f"Error fetching {origin} -> {destination}: {e}")
        return None

# -------------------------------
# LOOP OVER DESTINATIONS
# -------------------------------
results = []
output_file = 'airports_with_prices.csv'

for i, dest in enumerate(airports_subset['IATA']):
    if dest == ORIGIN_AIRPORT:
        continue

    print(f"Fetching {ORIGIN_AIRPORT} -> {dest} ({i+1}/{len(airports_subset)}) ...")
    price = get_cheapest_flight(ORIGIN_AIRPORT, dest, DEPARTURE_DATE)
    bucket, color = assign_bucket_and_color(price)

    results.append({
        'origin': ORIGIN_AIRPORT,
        'destination': dest,
        'price_usd': price,
        'bucket': bucket,
        'color': color
    })

    # Autosave
    if (i+1) % AUTOSAVE_INTERVAL == 0:
        df = pd.DataFrame(results)
        df.to_csv(output_file, index=False)
        print(f"Autosaved {i+1} airports to {output_file}")

    time.sleep(1)  # Prevent hitting rate limits

# -------------------------------
# FINAL SAVE
# -------------------------------
df = pd.DataFrame(results)
df.to_csv(output_file, index=False)
files.download(output_file)
print(f"Done! CSV saved as {output_file}")

Random subset of 1000 airports created.
Fetching MXP -> QVL (1/1000) ...
Error fetching MXP -> QVL: [400]

Fetching MXP -> BRT (2/1000) ...
Fetching MXP -> ZIV (3/1000) ...
Error fetching MXP -> ZIV: [400]

Fetching MXP -> HPV (4/1000) ...
Fetching MXP -> FOD (5/1000) ...
Fetching MXP -> MAD (6/1000) ...
Fetching MXP -> BVY (7/1000) ...
Error fetching MXP -> BVY: [400]

Fetching MXP -> KTD (8/1000) ...
Error fetching MXP -> KTD: [400]

Fetching MXP -> ELH (9/1000) ...
Fetching MXP -> SXS (10/1000) ...
Fetching MXP -> XNJ (11/1000) ...
Fetching MXP -> BPF (12/1000) ...
Error fetching MXP -> BPF: [400]

Fetching MXP -> JJI (13/1000) ...
Error fetching MXP -> JJI: [400]

Fetching MXP -> TFS (14/1000) ...
Fetching MXP -> DEA (15/1000) ...
Error fetching MXP -> DEA: [400]

Fetching MXP -> KGX (16/1000) ...
Error fetching MXP -> KGX: [400]

Fetching MXP -> IPN (17/1000) ...
Error fetching MXP -> IPN: [400]

Fetching MXP -> ILZ (18/1000) ...
Error fetching MXP -> ILZ: [400]

Fetching MXP -> V

In [18]:
!pip install amadeus

import datetime
from amadeus import Client, ResponseError

# -------------------------------
# CONFIG
# -------------------------------
API_KEY = "ligX17GEhAERS34XjfmMW6rpG1c5uHZu"       # Replace with your Amadeus GET API key
API_SECRET = "2jWVw0p9UA72Ykke" # Replace with your Amadeus secret
ORIGIN = "MXP"
DESTINATION = "JFK"
DEPARTURE_DATE = (datetime.date.today() + datetime.timedelta(days=30)).strftime("%Y-%m-%d")

# -------------------------------
# INIT CLIENT
# -------------------------------
amadeus = Client(client_id=API_KEY, client_secret=API_SECRET)

# -------------------------------
# PRICE BUCKET / COLOR FUNCTION
# -------------------------------
def assign_bucket_and_color(price):
    if price is None:
        return "No Data", "#CCCCCC"
    elif price < 100:
        return "<100", "#0000FF"        # Blue
    elif price < 200:
        return "100-200", "#00FF00"    # Green 1
    elif price < 400:
        return "200-400", "#008000"    # Green 2
    elif price < 800:
        return "400-800", "#FFFF00"    # Yellow
    elif price < 1000:
        return "800-1000", "#FFA500"   # Orange
    else:
        return ">1000", "#FF0000"      # Red

# -------------------------------
# FETCH CHEAPEST FLIGHT
# -------------------------------
def get_cheapest_flight(origin, destination, date):
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin,
            destinationLocationCode=destination,
            departureDate=date,
            adults=1,
            max=5
        )
        data = response.data
        if not data:
            return None
        prices = [float(offer['price']['total']) for offer in data if 'price' in offer]
        return min(prices) if prices else None
    except ResponseError as e:
        print(f"Error fetching {origin} -> {destination}: {e}")
        return None

# -------------------------------
# RUN
# -------------------------------
price = get_cheapest_flight(ORIGIN, DESTINATION, DEPARTURE_DATE)
bucket, color = assign_bucket_and_color(price)

print(f"{ORIGIN} -> {DESTINATION}:")
print(f"Price: {price}")
print(f"Bucket: {bucket}")
print(f"Color: {color}")

MXP -> JFK:
Price: 278.28
Bucket: 200-400
Color: #008000
